In [28]:
import tweepy
import yaml
import pandas as pd
import json
import os
import io
from collections import defaultdict

In [2]:
with open('config.yml', "r") as stream:
    try:
        cf = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)


auth = tweepy.OAuth1UserHandler(
    cf['twitter_key'], 
    cf['twitter_secret'], 
    cf['twitter_access_key'], 
    cf['twitter_access_secret']
)

In [27]:
user_list = [1643966166]

In [67]:
json.loads('last_tweet.json')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [72]:
# Load last tweet_id dict
if os.path.isfile('last_tweet.json') and os.access('last_tweet.json', os.R_OK):
    with open('last_tweet.json', 'r') as jsonfile:
        last_tweet_dict = json.load(jsonfile)
else:
    last_tweet_dict = defaultdict(lambda: None)

In [81]:
client = tweepy.Client(bearer_token=cf['twitter_bearer'])

def extract_tweet(id, since_id=None, pagination_token=None, first=True):
    res = client.get_users_tweets(id, since_id=since_id, tweet_fields=[
    'created_at', 
    'author_id', 
    'conversation_id', 
    'in_reply_to_user_id',
    'referenced_tweets',
    'attachments'],
    max_results=100,
    pagination_token=pagination_token)

    # Quit if no more data
    if res.data is None:
        return

    # Update last tweet id
    if first:
        print(id, res.data[0].id)
        last_tweet_dict[id] = res.data[0].id

    for curr in res.data:
        with open(f'tweet_json/{curr.data["id"]}.json', 'w') as json_file:
            json.dump(curr.data, json_file)
    
    if 'next_token' in res.meta:
        extract_tweet(id, pagination_token=res.meta['next_token'], first=False)

In [82]:
for user_id in user_list:
    extract_tweet(user_id, since_id=last_tweet_dict[str(user_id)])

with open('last_tweet.json', 'w') as json_file:
    json.dump(last_tweet_dict, json_file)

In [24]:



# @@bennstancil
res = client.get_users_tweets(id=1643966166, since_id=None, tweet_fields=[
    'created_at', 
    'author_id', 
    'conversation_id', 
    'in_reply_to_user_id',
    'referenced_tweets',
    'attachments'])


In [25]:
res

Response(data=[<Tweet id=1542942959913603072 text="@alexkyllo @matsonj @dataklump i think it goes well beyond this. it's not a semantic model of the *data* and their relationships; it's a operational model of the metrics and how they move one another. it's the company's supply and demand curves - an abstract description of how it moves.">, <Tweet id=1542938639486812160 text="@Cavorax @ArynnPost @leoebfolsom wait, that distinction doesn't make sense to me...what do you mean?">, <Tweet id=1542938367901536259 text='@alexkyllo @matsonj @dataklump a rant for another day, but i think it means that analysts do something different than just dashboards and decks, and instead build something like "operational models," which is an obnoxious term but a very valuable thing'>, <Tweet id=1542937549995446275 text='@gwenwindflower does this end up meaning we split analysts in half, and then the AE half conquers the analyst half, so we end up with analysts again, but this time everyone has appreciation 

In [20]:
res.data[0].data

{'text': "@alexkyllo @matsonj @dataklump i think it goes well beyond this. it's not a semantic model of the *data* and their relationships; it's a operational model of the metrics and how they move one another. it's the company's supply and demand curves - an abstract description of how it moves.",
 'author_id': '1643966166',
 'id': '1542942959913603072',
 'conversation_id': '1542911269803540480',
 'created_at': '2022-07-01T18:47:22.000Z',
 'in_reply_to_user_id': '1406821746',
 'referenced_tweets': [{'type': 'replied_to', 'id': '1542939515446210561'}]}

In [11]:
client.get_users_tweets(id=1643966166, pagination_token='7140dibdnow9c7btw421tf9cwn04vtmtaxevjgrmziu5s')

Response(data=[<Tweet id=1537857493224697856 text="@juansequeda @HonestNoBSData i didn't, i avoided it and every booth within 50 feet of it">, <Tweet id=1537857146011803648 text='@gwenwindflower definitely read that as https://t.co/NIBJPHdEry'>, <Tweet id=1537856769027846144 text="@JPedro_Monteiro @decentriq no, different thing. I don't know exactly what the company did to be honest. It was just enough to get me to realize what other companies could do.">, <Tweet id=1537843768279199744 text="it's friday, let's fight snowflake about what snowflake is building\n\nhttps://t.co/aZ9tRcVZOU">, <Tweet id=1536357557329084416 text='@thenanyu i hear they got a good designer tho'>, <Tweet id=1536073886701330436 text='@Patrick_Rankin better that than the scary magic the gathering teenagers at joe muggs that you had to walk past to get to the people playing pokemon'>, <Tweet id=1536073217294614530 text="@mullinsms They could, I suppose, but don't seem to have that much interest in it">, <Tweet id=1

In [10]:
type(res.data[0])

tweepy.tweet.Tweet